In [1]:
# Load feature groups from Json file
import json

with open("feature_groups.json", "r") as f:
    good_bad_dict = json.load(f)

good_features = good_bad_dict["good_features"]
bad_features = good_bad_dict["bad_features"]

In [2]:
# Load test data

import pandas as pd
import numpy as np

data = pd.read_csv('../data/investigation_train_large_checked.csv')

# Let's specify the features and the target
y = data['checked']
X = data.drop(columns=['checked', 'Ja', 'Nee'])
X = X.astype(np.float32)


C:\Users\yutin\AppData\Local\Temp\ipykernel_14872\1726338827.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [20]:
# Specify model paths

model_2 = '../model/bad_model_histGradBoosting.onnx'
model_1 = '../model/model_1.onnx'

In [21]:
# Load ONNX models

import onnxruntime as rt

sess1 = rt.InferenceSession(model_1)
sess2 = rt.InferenceSession(model_2)

In [5]:
# Perturbation function

import random

def metamorphic_transform_value(value, unique_values, num_unique_values, categorical_threshold=100):
    """
    Metamorphic transformation for a feature value.
    If the feature is binary (2 unique values), flip the value.
    If the feature is categorical with more than 2 unique values, randomly select a different value.
    If the feature is continuous (more than categorical_threshold unique values), perturb the value.
    Parameters:
    - value: original feature value
    - unique_values: list of unique values for the feature
    - num_unique_values: number of unique values for the feature
    - categorical_threshold: threshold to distinguish categorical from continuous features
    Returns:
    - transformed_value: metamorphically transformed feature value
    """
    
    if num_unique_values <= 2:
        # Binary feature, flip the value
        if value in [0, 1]:
            return 1 - value
        return value  # In case of unexpected binary values
    
    elif num_unique_values <= categorical_threshold:
        # Categorical feature with more than 2 unique values
        other_values = [v for v in unique_values if v != value]
        if len(other_values) == 0:
            return value  # No other value to choose from
        return random.choice(other_values)
    
    else:
        # Continuous feature, perturb the value slightly
        min_value = min(unique_values)
        max_value = max(unique_values)
        # Bounded perturbation
        delta = (max_value - min_value) * 0.05  # 5% of the range
        perturbed_value = value + random.uniform(-delta, delta)
        # Ensure perturbed value is within the original range
        perturbed_value = max(min_value, min(max_value, perturbed_value))
        return perturbed_value

In [8]:
# # Build dictionary of unique values and counts for each bad feature
bad_feature_unique_values = {}
for feature in bad_features:
    unique_values = X[feature].unique().tolist()
    bad_feature_unique_values[feature] = {
        'unique_values': unique_values,
        'num_unique_values': len(unique_values)
    }

### Test 1: Perturb one feature and test number of prediction changes

In [6]:
def metamorphic_feature_test(X_test, y_test, feature, feature_unique_values, feature_unique_counts,
                             session, input_name):
    
    total_rows = len(X_test)

    X_test_np = X_test.to_numpy().astype(np.float32)
    pred_original = session.run(None, {input_name: X_test_np})[0]

    X_test_flipped = X_test.copy()
    X_test_flipped[feature] = X_test_flipped[feature].apply(
        lambda val: metamorphic_transform_value(
            val,
            feature_unique_values[feature],
            feature_unique_counts[feature]
        )
    )

    X_test_flipped_np = X_test_flipped.to_numpy().astype(np.float32)

    pred_transformed = session.run(None, {input_name: X_test_flipped_np})[0]

    # Print how many predictions' labels are "True"'
    print(
        f"Feature '{feature}': Original 'True' predictions: {np.sum(pred_original == True)}, "
        f"Transformed 'True' predictions: {np.sum(pred_transformed == True)}"
    )

    # Evaluate accuracy
    accuracy_original = np.mean(pred_original == y_test)
    accuracy_transformed = np.mean(pred_transformed == y_test)

    changed_predictions = np.sum(pred_original != pred_transformed)

    result = {
        "feature": feature,
        "total_rows": total_rows,
        "changed_predictions": changed_predictions,
        "changed_percentage": changed_predictions / total_rows,
        "accuracy_original": accuracy_original,
        "accuracy_transformed": accuracy_transformed

    }

    return result


In [22]:
# Apply the test

results_model_1 = []
input_name1 = sess1.get_inputs()[0].name
for feature in bad_features:
    res = metamorphic_feature_test(
        X_test=X,
        y_test=y,
        feature=feature,
        feature_unique_values={f: bad_feature_unique_values[f]['unique_values'] for f in bad_features},
        feature_unique_counts={f: bad_feature_unique_values[f]['num_unique_values'] for f in bad_features},
        session=sess1,
        input_name=input_name1
    )
    results_model_1.append(res)

results_model_2 = []
input_name2 = sess2.get_inputs()[0].name
for feature in bad_features:
    res = metamorphic_feature_test(
        X_test=X,
        y_test=y,
        feature=feature,
        feature_unique_values={f: bad_feature_unique_values[f]['unique_values'] for f in bad_features},
        feature_unique_counts={f: bad_feature_unique_values[f]['num_unique_values'] for f in bad_features},
        session=sess2,
        input_name=input_name2
    )
    results_model_2.append(res)
    

Feature 'afspraak_aantal_woorden': Original 'True' predictions: 17248, Transformed 'True' predictions: 17227
Feature 'afspraak_laatstejaar_aantal_woorden': Original 'True' predictions: 17248, Transformed 'True' predictions: 17245
Feature 'belemmering_dagen_psychische_problemen': Original 'True' predictions: 17248, Transformed 'True' predictions: 17234
Feature 'belemmering_hist_psychische_problemen': Original 'True' predictions: 17248, Transformed 'True' predictions: 17549
Feature 'belemmering_hist_taal': Original 'True' predictions: 17248, Transformed 'True' predictions: 17248
Feature 'belemmering_psychische_problemen': Original 'True' predictions: 17248, Transformed 'True' predictions: 17052
Feature 'competentie_ethisch_en_integer_handelen': Original 'True' predictions: 17248, Transformed 'True' predictions: 17248
Feature 'competentie_gedrevenheid_en_ambitie_tonen': Original 'True' predictions: 17248, Transformed 'True' predictions: 17290
Feature 'competentie_met_druk_en_tegenslag_omg

In [23]:
import pandas as pd

df1 = pd.DataFrame(results_model_1)
df2 = pd.DataFrame(results_model_2)

df1.to_csv("model_1_results.csv", index=False)
df2.to_csv("model_2_results.csv", index=False)


In [24]:
# Compare two models' results 

def compare_models(results_A, results_B):
    # Convert lists to feature-indexed dicts
    A = {r["feature"]: r for r in results_A}
    B = {r["feature"]: r for r in results_B}

    stability_A = 0
    stability_B = 0
    acc_drop_A = 0
    acc_drop_B = 0

    comparison = []

    for f in A.keys():
        change_A = A[f]["changed_percentage"]
        change_B = B[f]["changed_percentage"]

        accA_drop = A[f]["accuracy_original"] - A[f]["accuracy_transformed"]
        accB_drop = B[f]["accuracy_original"] - B[f]["accuracy_transformed"]

        stability_A += change_A
        stability_B += change_B
        acc_drop_A += accA_drop
        acc_drop_B += accB_drop

        comparison.append({
            "feature": f,
            "change_A": change_A,
            "change_B": change_B,
            "difference": change_B - change_A,
            "acc_drop_A": accA_drop,
            "acc_drop_B": accB_drop
        })
    
    fairness_score_A = stability_A + acc_drop_A
    fairness_score_B = stability_B + acc_drop_B

    return {
        "per_feature_comparison": comparison,
        "global": {
            "stability_A": stability_A,
            "stability_B": stability_B,
            "acc_drop_A": acc_drop_A,
            "acc_drop_B": acc_drop_B,
            "fairness_score_A": fairness_score_A,
            "fairness_score_B": fairness_score_B,
            "best_model": "A" if fairness_score_A < fairness_score_B else "B"
        }
    }



In [25]:
# Example usage of the metamorphic transformation comparison
comparison_result = compare_models(results_model_1, results_model_2)
comparison_result['global']

{'stability_A': 0.3530538461538461,
 'stability_B': 0.28163846153846156,
 'acc_drop_A': 0.158992307692307,
 'acc_drop_B': 0.12528461538461466,
 'fairness_score_A': 0.5120461538461532,
 'fairness_score_B': 0.4069230769230762,
 'best_model': 'B'}

In [26]:
# Save comparison result to JSON
with open("model_comparison.json", "w") as f:
    json.dump(
        comparison_result,
        f,
        indent=4
    )